<a href="https://colab.research.google.com/github/slachitoff/CS-GY-6613-Assignments/blob/main/DeepLearning_MidTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, Dataset

from torchvision.transforms import ToPILImage

import numpy as np
import pickle

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10_batch(batch_filename):
    batch = unpickle(batch_filename)
    images = batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    labels = batch[b'labels']
    return images, labels

train_images1, train_labels1 = load_cifar10_batch('/content/data_batch_1')
train_images2, train_labels2 = load_cifar10_batch('/content/data_batch_2')
train_images3, train_labels3 = load_cifar10_batch('/content/data_batch_3')
train_images4, train_labels4 = load_cifar10_batch('/content/data_batch_4')
train_images5, train_labels5 = load_cifar10_batch('/content/data_batch_5')

original_train_images = np.concatenate((train_images1, train_images2, train_images3, train_images4, train_images5))
original_train_labels = np.concatenate((train_labels1, train_labels2, train_labels3, train_labels4, train_labels5))

transform_train = transforms.Compose([
    ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_train = transforms.Compose([
    ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

num_train = int(0.95 * len(original_train_images))
num_val = len(original_train_images) - num_train

shuffled_indices = np.random.permutation(len(original_train_images))

train_idx = shuffled_indices[:num_train]
val_idx = shuffled_indices[num_train:]

train_images_subset = original_train_images[train_idx]
val_images_subset = original_train_images[val_idx]
train_labels_subset = original_train_labels[train_idx]
val_labels_subset = original_train_labels[val_idx]

class CIFAR10Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label

BATCH_SIZE_TRAIN = 128
BATCH_SIZE_TEST = 100

train_dataset = CIFAR10Dataset(train_images_subset, train_labels_subset, transform=transform_train)
val_dataset = CIFAR10Dataset(val_images_subset, val_labels_subset, transform=transform_test)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=2)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE_TEST, shuffle=False, num_workers=2)


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_rate=0.5):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.dropout_final = nn.Dropout(dropout_rate)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.dropout1(out)

        out = self.layer2(out)
        out = self.dropout2(out)

        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)

        out = self.dropout_final(out)
        out = self.linear(out)
        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ResNet(BasicBlock, [2, 1, 1, 1], num_classes=10, dropout_rate=0.4)
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [ ]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
          Dropout-13           [-1, 64, 32, 32]               0
           Conv2d-14          [-1, 128,

In [ ]:
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_epoch(model, optimizer, scheduler, data_loader, criterion, device, loss_history):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = 100. * correct / total
    print(f'Training: Average Loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({accuracy:.0f}%)')
    loss_history.append(avg_loss)

    return avg_loss, accuracy

def validate(model, data_loader, criterion, device):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()

    avg_loss = val_loss / len(data_loader)
    accuracy = 100. * correct / len(data_loader.dataset)
    print(f'Validation: Average Loss: {avg_loss:.4f}, Accuracy: {correct}/{len(data_loader.dataset)} ({accuracy:.0f}%)')

    return avg_loss, accuracy

N_EPOCHS = 200
optimizer = torch.optim.AdamW(model.parameters(), lr=0.002, weight_decay=0.02)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=N_EPOCHS)
criterion = nn.CrossEntropyLoss()

start_time = time.time()
train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []

for epoch in range(1, N_EPOCHS + 1):
    print(f'\nEpoch {epoch}/{N_EPOCHS}')
    train_avg_loss, train_accuracy = train_epoch(model, optimizer, scheduler, train_loader, criterion, device, train_loss_history)
    train_accuracy_history.append(train_accuracy)

    val_avg_loss, val_accuracy = validate(model, val_loader, criterion, device)
    val_loss_history.append(val_avg_loss)
    val_accuracy_history.append(val_accuracy)

execution_time = time.time() - start_time
print(f'\nExecution time: {execution_time:.2f} seconds')


Epoch 1/200
Training: Average Loss: 1.6801, Accuracy: 17456/47500 (37%)
Validation: Average Loss: 1.4920, Accuracy: 1148/2500 (46%)

Epoch 2/200
Training: Average Loss: 1.3651, Accuracy: 23783/47500 (50%)
Validation: Average Loss: 1.2957, Accuracy: 1343/2500 (54%)

Epoch 3/200
Training: Average Loss: 1.2079, Accuracy: 26836/47500 (56%)
Validation: Average Loss: 1.1825, Accuracy: 1449/2500 (58%)

Epoch 4/200
Training: Average Loss: 1.1203, Accuracy: 28502/47500 (60%)
Validation: Average Loss: 1.1261, Accuracy: 1509/2500 (60%)

Epoch 5/200
Training: Average Loss: 1.0424, Accuracy: 29900/47500 (63%)
Validation: Average Loss: 1.1852, Accuracy: 1532/2500 (61%)

Epoch 6/200
Training: Average Loss: 0.9998, Accuracy: 30545/47500 (64%)
Validation: Average Loss: 1.0419, Accuracy: 1591/2500 (64%)

Epoch 7/200
Training: Average Loss: 0.9548, Accuracy: 31369/47500 (66%)
Validation: Average Loss: 0.9852, Accuracy: 1686/2500 (67%)

Epoch 8/200
Training: Average Loss: 0.9241, Accuracy: 31829/47500 (6

In [ ]:
from PIL import Image

test_images, test_labels = load_cifar10_batch('/content/test_batch')

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

test_images_tensor = torch.stack([transform_test(Image.fromarray(img)) for img in test_images])
test_labels_tensor = torch.tensor(test_labels)

test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100.0 * correct / total
print(f'Accuracy on the test set: {test_accuracy}%')

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize

transform_test = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_pickle_file(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data

unlabeled_test_data = load_pickle_file('/content/cifar_test_nolabels.pkl')
num_images = unlabeled_test_data[b'data'].shape[0]
test_images = unlabeled_test_data[b'data'].reshape((num_images, 3, 32, 32)).transpose((0, 2, 3, 1))

transformed_images = [transform_test(Image.fromarray(img)) for img in test_images]

predictions = []

with torch.no_grad():
    for i, img in enumerate(transformed_images):
        img = img.unsqueeze(0).to(device)
        outputs = model(img)
        _, predicted = torch.max(outputs, 1)
        predictions.append((i, predicted.item()))

df_predictions = pd.DataFrame(predictions, columns=["ID", "Labels"])
df_predictions.to_csv('/content/predicted_labels.csv', index=False)
